In [1]:
import os
import time

import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
path = 'data'
train_path = os.path.join(path, 'train')
val_path = os.path.join(path, 'val')
test_path = os.path.join(path, 'test')
print(11)

11


In [3]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_dataset = datasets.ImageFolder(train_path, transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]))
valid_dataset = datasets.ImageFolder(val_path, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))
print(len(train_dataset))

6516


In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    num_workers=4
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=64,
    num_workers=4
)

In [ ]:
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
       
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.conv1_bn = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2, 2)
       
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.conv2_bn = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3_bn = nn.BatchNorm2d(64)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3)
        self.conv4_bn = nn.BatchNorm2d(128)
        self.pool4 = nn.MaxPool2d(2, 2)
        
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3)
        self.conv5_bn = nn.BatchNorm2d(256)
        self.pool5 = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(256 * 5 * 5, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        in_size = x.size(0)
        x = self.pool(F.relu(self.conv1_bn(self.conv1(x))))
        x = self.pool2(F.relu(self.conv2_bn(self.conv2(x))))
        x = self.pool3(F.relu(self.conv3_bn(self.conv3(x))))
        x = self.pool4(F.relu(self.conv4_bn(self.conv4(x))))
        x = self.pool5(F.relu(self.conv5_bn(self.conv5(x))))
        x = x.view(-1, 256*5*5)
        x = F.dropout(F.relu(self.fc1(x)), training=self.training, p=0.4)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [5]:
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
       
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5)     
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.conv2_bn = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)     
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3)
        self.conv4_bn = nn.BatchNorm2d(128)
        self.pool4 = nn.MaxPool2d(2, 2)
        
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3)
        self.conv6 = nn.Conv2d(256, 512, kernel_size=3)
        self.conv6_bn = nn.BatchNorm2d(512)
        self.pool6 = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(512 * 26 * 26, 2048)
        self.fc2 = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        in_size = x.size(0)
        x = self.conv1(x)
        x = self.pool2(F.relu(self.conv2_bn(self.conv2(x))))
        
        x = self.conv3(x)
        x = self.pool4(F.relu(self.conv4_bn(self.conv4(x))))
        
        x = self.conv5(x)
        x = self.pool6(F.relu(self.conv6_bn(self.conv6(x))))
        
        x = x.view(-1, 512*26*26)
        x = F.dropout(F.relu(self.fc1(x)), training=self.training, p=0.4)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [11]:
model = Net(196)
model.to(device)
from torchsummary import summary

summary(model, (3, 224, 224))

RuntimeError: shape '[-1, 346112]' is invalid for input of size 589824

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum=0.9)
from torch.optim.lr_scheduler import *
scheduler=StepLR(optimizer,step_size=3)

In [ ]:
def train(model,device, train_loader, epoch):
    model.train()
    for batch_idx, data in enumerate(train_loader):
        x,y= data
        x=x.to(device)
        y=y.to(device)
        optimizer.zero_grad()
        y_hat= model(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
    print ('Train Epoch: {}\t Loss: {:.6f}'.format(epoch,loss.item()))

In [ ]:
def valid(model, device, valid_loader):
    model.eval()
    valid_loss = 0
    correct = 0
    with torch.no_grad():
        for i,data in enumerate(valid_loader):          
            x,y= data
            x=x.to(device)
            y=y.to(device)
            optimizer.zero_grad()
            y_hat = model(x)
            valid_loss += criterion(y_hat, y).item() # sum up batch loss
            pred = y_hat.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()
    valid_loss /= len(valid_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        valid_loss, correct, len(valid_dataset),
        100. * correct / len(valid_dataset)))

In [ ]:
for epoch in range(1, 100):
    train(model=model, device=device, train_loader=train_loader, epoch=epoch)
    valid(model=model, device=device, valid_loader=valid_loader)